# adj_list = [relation1, relation2, relation3]

In [9]:
import scipy.sparse as sp
import numpy as np
from collections import defaultdict


# 随机生成一个10x10的邻接矩阵，其中非零元素的概率为0.3
adjacency_matrix_1 = np.random.choice([0, 1], size=(10, 10), p=[0.7, 0.3])
adjacency_matrix_1 = np.tril(adjacency_matrix_1) + np.tril(adjacency_matrix_1, -1).T
print(adjacency_matrix_1)
adjacency_matrix_2 = np.random.choice([0, 1], size=(10, 10), p=[0.7, 0.3])
adjacency_matrix_2 = np.tril(adjacency_matrix_2) + np.tril(adjacency_matrix_2, -1).T
adjacency_matrix_3 = np.random.choice([0, 1], size=(10, 10), p=[0.7, 0.3])
adjacency_matrix_3 = np.tril(adjacency_matrix_3) + np.tril(adjacency_matrix_3, -1).T

# 将邻接矩阵转换为稀疏矩阵格式
sparse_matrix_1 = sp.csr_matrix(adjacency_matrix_1)
sparse_matrix_2 = sp.csr_matrix(adjacency_matrix_2)
sparse_matrix_3 = sp.csr_matrix(adjacency_matrix_3)

temp_array = [sparse_matrix_1, sparse_matrix_2, sparse_matrix_3]
adj_lists = []

for relation in temp_array:
    homo_adj = relation + sp.eye(relation.shape[0])
    # create adj_list
    adj_li = defaultdict(set)
    edges = homo_adj.nonzero()
    for index, node in enumerate(edges[0]):
        adj_li[node].add(edges[1][index])
        adj_li[edges[1][index]].add(node)
    adj_lists.append(adj_li)

print(adj_lists)

[[1 1 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 1 0 0 0 0]
 [1 0 1 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 1 0]]
[defaultdict(<class 'set'>, {0: {0, 1, 2}, 1: {0, 1, 5}, 2: {0, 2, 5, 6}, 5: {1, 2, 5, 9}, 6: {2, 6}, 3: {3}, 4: {4}, 9: {8, 9, 5}, 7: {7}, 8: {8, 9}}), defaultdict(<class 'set'>, {0: {0, 2, 3, 4}, 2: {0, 1, 2, 9}, 3: {0, 3, 4, 7}, 4: {0, 1, 3, 4, 8}, 1: {1, 2, 4}, 9: {9, 2, 7}, 7: {3, 6, 7, 8, 9}, 8: {8, 4, 7}, 5: {5, 6}, 6: {5, 6, 7}}), defaultdict(<class 'set'>, {0: {0, 9, 5, 7}, 5: {0, 4, 5}, 7: {0, 1, 4, 7}, 9: {0, 9, 4, 6}, 1: {1, 6, 7}, 6: {1, 2, 4, 6, 8, 9}, 2: {2, 3, 6}, 3: {2, 3}, 4: {4, 5, 6, 7, 9}, 8: {8, 6}})]


# InterAgg

## forward

In [11]:
to_neighs = []
nodes = [0, 1, 2]
# 对adj_list循环提取每一种关系，
for adj_list in adj_lists:
    to_neighs.append([set(adj_list[int(node)]) for node in nodes])

to_neighs

[[{0, 1, 2}, {0, 1, 5}, {0, 2, 5, 6}],
 [{0, 2, 3, 4}, {1, 2, 4}, {0, 1, 2, 9}],
 [{0, 5, 7, 9}, {1, 6, 7}, {2, 3, 6}]]

上述代码每一个元素都是一个一个关系的节点邻居

In [15]:
unique_nodes = set.union(set.union(*to_neighs[0]), set.union(*to_neighs[1]),
                         set.union(*to_neighs[2], set(nodes)))
unique_nodes

{0, 1, 2, 3, 4, 5, 6, 7, 9}

返回所有邻居的节点标号（索引）

In [ ]:
batch_features = self.features(torch.cuda.LongTensor(list(unique_nodes)))

提取所有unique_nodes的feature